In [ ]:
with open("input.txt") as f:
    raw = f.read().splitlines()

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Callable, Optional


@dataclass
class Monkey:
    starting: list[int] = field(default_factory=list)
    operation: Callable[[int], int] = lambda _: _
    test: Callable[[int], int] = lambda _:_
    if_true: int = -1
    if_false: int = -1
    inspects: int = 0

In [ ]:
monkey_regex = [
    "Monkey (\\d+):",
    "(\\d+)",
    "Operation: (.+)",
    "Test: (.+)",
    "If true: throw to monkey (\\d+)",
    "If false: throw to monkey (\\d+)"
]


In [ ]:
import math
import re


monkeys: dict[int, Monkey] = {}

monkey = 0
line_parsed = 0
rounds = 0

def operation_to_lambda(string: str):
    op = string.split(" ")
    print(op[3], op[4])
    if op[3] == "*":
        if op[4] == "old":
            return lambda x: x ** 2
        return lambda x: x * int(op[4])
    if op[3] == "/":
        return lambda x: x // int(op[4])
    if op[3] == "+":
        return lambda x: x + int(op[4])
    if op[3] == "-":
        return lambda x: x - int(op[4])
    raise ArithmeticError()

def test_to_lambda(string: str):
    op = string.split(" ")
    return lambda x: not bool(x % int(op[2]))

for line in raw:
    if not line:
        line_parsed = 0
    else:
        rg = monkey_regex[line_parsed]
        if line_parsed == 0:
            monkey = int(re.compile(rg).findall(line)[0])
            monkeys[monkey] = Monkey()
        elif line_parsed == 1:
            for i in re.compile(rg).findall(line):
                monkeys[monkey].starting.append(int(i))
        elif line_parsed == 2:
            monkeys[monkey].operation = operation_to_lambda(re.compile(rg).findall(line)[0])
        elif line_parsed == 3:
            monkeys[monkey].test = test_to_lambda(re.compile(rg).findall(line)[0])
        elif line_parsed == 4:
            monkeys[monkey].if_true = int(re.compile(rg).findall(line)[0])
        elif line_parsed == 5:
            monkeys[monkey].if_false = int(re.compile(rg).findall(line)[0])
        line_parsed += 1

print(monkeys)

In [ ]:
r = 20
def do_round(times: int):
    for _ in range(times):
        for index, monkey in monkeys.items():
            while monkey.starting:
                item = monkey.starting.pop(0)
                monkey.inspects += 1
                item = monkey.operation(item)
                item = item//3
                if monkey.test(item):
                    monkeys[monkey.if_true].starting.append(item)
                else:
                    monkeys[monkey.if_false].starting.append(item)
rounds += r
do_round(r)
print({rounds: {i: (m.inspects, m.starting) for i, m in monkeys.items()}})
m = sorted(monkeys.values(), key=lambda x: x.inspects, reverse=True)
print(list((m.inspects, m.starting) for m in m))
print(f"Business: {m[0].inspects*m[1].inspects}")

In [ ]:
r = 10000 - rounds
def do_round(times: int):
    for i in range(times):
        print(f"Iteration {i}")
        for index, monkey in monkeys.items():
            while monkey.starting:
                item = monkey.starting.pop(0)
                monkey.inspects += 1
                item = monkey.operation(item)
                if monkey.test(item):
                    monkeys[monkey.if_true].starting.append(item)
                else:
                    monkeys[monkey.if_false].starting.append(item)
rounds += r
do_round(r)
print({rounds: {i: (m.inspects, m.starting) for i, m in monkeys.items()}})
m = sorted(monkeys.values(), key=lambda x: x.inspects, reverse=True)
print(list((m.inspects, m.starting) for m in m))
print(f"Business: {m[0].inspects*m[1].inspects}")